<a href="https://colab.research.google.com/github/vimesh630/Revenue_Forecasting/blob/main/Feature_Engineering_for_Revenue_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import os